In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/search-data-1/data.json
/kaggle/input/search-data/questions.json
/kaggle/input/search-data/sentences.json


In [20]:
!pip install faiss-gpu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [25]:
import json

with open('/kaggle/input/search-data-1/data.json','r') as file:
    data=json.load(file)

with open('/kaggle/input/search-data/questions.json','r') as file:
    questions=json.load(file)


In [4]:
data[0]

{'title': 'Pandemic',
 'text': 'A pandemic (from Greek πᾶν, pan, "all" and δῆμος, demos, "people") is an epidemic of an infectious disease that has spread across a large region, for instance multiple continents or worldwide, affecting a substantial number of people. A widespread endemic disease with a stable number of infected people is not a pandemic. Widespread endemic diseases with a stable number of infected people such as recurrences of seasonal influenza are generally excluded as they occur simultaneously in large regions of the globe rather than being spread worldwide.\nThroughout human history, there have been a number of pandemics of diseases such as smallpox and tuberculosis. The most fatal pandemic in recorded history was the Black Death (also known as The Plague), which killed an estimated 75–200 million people in the 14th century. The term was not used yet but was for later pandemics including the 1918 influenza pandemic (Spanish flu). Current pandemics include COVID-19 (S

In [5]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=a1729ed2df7044da6e454027b12d0f841692a458798a25a3e7f790ae3fc527fb
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [6]:
from sentence_transformers import SentenceTransformer,util

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [8]:
model=SentenceTransformer('msmarco-distilbert-base-v3',device='cuda')

In [14]:
a=model.encode(data[0]['text'])
a.shape

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

(768,)

In [91]:
sentence_embeddings=[model.encode(a['text']) for a in data]
sentence_embeddings_norm=[arr/np.linalg.norm(arr) for arr in sentence_embeddings]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [17]:
sentence_embeddings[0].shape

(768,)

In [93]:
import torch
import faiss
d = 768
index = faiss.IndexFlatIP(d)

In [94]:
index.add(np.array(sentence_embeddings_norm))

In [101]:
def search_func(question,k):
    question_vector=model.encode(question)
    question_vector_norm=question_vector/np.linalg.norm(question_vector)
    Distances,Indexes=index.search(question_vector_norm.reshape(1,d),k)
    print(*Distances)
    print(*Indexes)
    docs=[data[a]['text'] for a in Indexes[0]]
    return {question:dict(zip(docs,Distances[0]))}
    

In [108]:
search_func(questions[0],5)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[0.2682636  0.26307985 0.22907783 0.21605128 0.20000796]
[19  1  2  0  4]


{'How many people have died during Black Death?': {'The Spanish flu, also known as the 1918 flu pandemic, was an unusually deadly influenza pandemic caused by the H1N1 influenza A virus. Lasting from February 1918 to April 1920, it infected 500 million people – about a third of the world\'s population at the time – in four successive waves. The death toll is typically estimated to have been somewhere between 17 million and 50 million, and possibly as high as 100 million, making it one of the deadliest pandemics in human history.The first observations of illness and mortality were documented in the United States (in Fort Riley, Haskell County, Kansas as well as in New York City), France (Brest), Germany and the United Kingdom. To maintain morale, World War I censors minimized these early reports. Newspapers were free to report the epidemic\'s effects in neutral Spain, such as the grave illness of King Alfonso XIII, and these stories created a false impression of Spain as especially hard

In [109]:
faiss.write_index(index, "index_faiss_sbert.index")

In [78]:
import numpy as np
question_vector=model.encode(questions[0])
key_vector=model.encode(data[0]['text'])

def cos_sim(u,v):
    dot_product=np.dot(u,v)
    u_mod=np.sqrt((u*u).sum())
    v_mod=np.sqrt((v*v).sum())
    return dot_product/(u_mod*v_mod)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]